# Task2 & Task 3

## 1. Download data and do Data Cleaning

In [13]:
import numpy as np
import pandas as pd
import seaborn as sns

In [14]:
# Loads the food balance data & item group data 
total = pd.read_csv("food_balance_total.csv",encoding = "ISO-8859-1", engine='python')
item_group = pd.read_csv("food_balance_item_group.csv")
total=total.merge(item_group, on='Item Code', how='left')
animal=total.loc[total['Item Group Code'] ==2941]
vegetal=total.loc[total['Item Group Code'] ==2903]
animal.to_csv('food_balance_animal.csv')
vegetal.to_csv('food_balance_vegetal.csv')
ani = pd.read_csv("food_balance_animal.csv")
veg = pd.read_csv("food_balance_vegetal.csv")
ani=ani.drop(['Unnamed: 0','Item Group Code', 'Item Group', 'Item_y', 'Factor', 'HS Code','HS07 Code', 'HS12 Code', 'CPC Code'], axis=1)
veg=veg.drop(['Unnamed: 0','Item Group Code', 'Item Group', 'Item_y', 'Factor', 'HS Code','HS07 Code', 'HS12 Code', 'CPC Code'], axis=1)
# Adds variable ‘origin’
veg["origin"] = "vegetal"
ani["origin"] = "animal"

In [15]:
# Adds variable ‘origin’
veg["origin"] = "vegetal"
ani["origin"] = "animal"
# Appends veg and ani to one dataframe
temp = ani.append(veg)
# Deletion of ani and veg file
del ani, veg
# Renaming of temp’s columns
temp.columns = ["country_code", "country",'item_code','item','x1','element','x2','year','unit','value','x3','origin']
# Transformation of ‘temp’ to a pivot table
data = temp.pivot_table(
index=["country_code", "country", "item_code", "item", "year", "origin"],
columns = ["element"], values=["value"], aggfunc=sum)


In [16]:
# Renaming of data’s columns (be careful, the order of the columns needs
# to be adapted to your own data !)
data.columns = ['domestic_supply_quantity', 'export_quantity','fat_supply_quantity_gcapitaday','feed',
'food', 'food_supply_kcalcapitaday','food_supply_quantity_kgcapitayr', 'import_quantity','losses', 'other_uses',
'processing', 'production', 'protein_supply_quantity_gcapitaday','residuals','seed', 'stock_variation', 'tourist_consumption']
# Index columns need to be normal columns
data = data.reset_index()
data.columns
data.head()

,country_code,country,item_code,item,year,origin,domestic_supply_quantity,export_quantity,fat_supply_quantity_gcapitaday,feed,...,import_quantity,losses,other_uses,processing,production,protein_supply_quantity_gcapitaday,residuals,seed,stock_variation,tourist_consumption
0,1,Armenia,2511,Wheat and products,2014,vegetal,639.000000,1.000000,3.900000,110.000000,...,344.000000,30.000000,0.000000,16.000000,338.000000,33.010000,46.000000,32.000000,42.000000,nan
1,1,Armenia,2511,Wheat and products,2015,vegetal,748.000000,2.000000,4.020000,125.000000,...,324.000000,59.000000,0.000000,16.000000,363.000000,33.070000,106.000000,33.000000,-63.000000,nan
2,1,Armenia,2511,Wheat and products,2016,vegetal,650.000000,1.000000,4.220000,128.000000,...,312.000000,64.000000,0.000000,17.000000,350.000000,34.700000,-22.000000,33.000000,11.000000,nan
3,1,Armenia,2511,Wheat and products,2017,vegetal,547.000000,1.000000,4.280000,105.000000,...,337.000000,15.000000,0.000000,16.000000,176.000000,34.360000,-42.000000,25.000000,-34.000000,nan
4,1,Armenia,2511,Wheat and products,2018,vegetal,571.000000,1.000000,4.400000,118.000000,...,379.000000,16.000000,0.000000,16.000000,187.000000,34.250000,-27.000000,20.000000,-5.000000,nan


## 2. Equation example: Wheat in France

In [19]:
#Domestic supply quantity
#Production+   Import Quantity-  Stock Variation- Export Quantity
#Feed+seed+Losses+Processing+Other Uses+Residual+Food

## 3. Popolulation calculation

In [52]:
population = total [total.Flag != 'A']
population = population.rename(columns = {'Area Code': 'Area_Code'})
# Get rid of data on China using area code 351
population= population[population.Area_Code!= 351]
#population= population[population.Year== 2014]
population= population[population.Element== 'Total Population - Both sexes']
population=population[['Area', 'Value','Year']]
population['Value']=population['Value']*1000
population = population.rename(columns = {'Area': 'country', 'Value': 'population','Year':'year'}, inplace = False)
population_uncleaned=population

population=population.groupby('year')['population'].sum()
population=population.reset_index()


In [57]:
population.head()

,year,population
0,2014,7131485000.000000
1,2015,7212776000.000000
2,2016,7293574000.000000
3,2017,7373798000.000000
4,2018,7453133000.000000


# Task4

In [18]:
#food_supply_kcal,food_supply_kgprotein,food_supply_kg,ratio_kcalkg,protein_percentage,dom_sup_kcal,dom_sup_kgprot,great_import_from_undern_countries

df=pd.merge(data,population,on='country')
pd.set_option('float_format', '{:f}'.format)
df['food_supply_kcal']=df['food_supply_kcalcapitaday']*df['population']*365
df['food_supply_kgprotein']=df['protein_supply_quantity_gcapitaday']*365*0.001*df['population']
df['food_supply_kg']=df['food']*1000000
df['ratio_kcalkg']=df['food_supply_kcal']/df['food_supply_kg']
df['protein_percentage']=df['food_supply_kgprotein']/df['food_supply_kg']
df['dom_sup_kcal']=df['ratio_kcalkg']*df['domestic_supply_quantity']*1000000
df['dom_sup_kgprot']=df['protein_percentage']*df['domestic_supply_quantity']*1000000
df=df.fillna(0)
df.head()

,country_code,country,item_code,item,year_x,origin,domestic_supply_quantity,export_quantity,fat_supply_quantity_gcapitaday,feed,...,tourist_consumption,population,year_y,food_supply_kcal,food_supply_kgprotein,food_supply_kg,ratio_kcalkg,protein_percentage,dom_sup_kcal,dom_sup_kgprot
0,1,Armenia,2511,Wheat and products,2014,vegetal,639.000000,1.000000,3.900000,110.000000,...,0.000000,2912000.000000,2014,1176608160000.000000,35085668.800000,406000000.000000,2898.049655,0.086418,1851853729655.172119,55221040.303448
1,1,Armenia,2511,Wheat and products,2015,vegetal,748.000000,2.000000,4.020000,125.000000,...,0.000000,2912000.000000,2014,1178733920000.000000,35149441.600000,409000000.000000,2881.990024,0.085940,2155728538288.508545,64283086.349144
2,1,Armenia,2511,Wheat and products,2016,vegetal,650.000000,1.000000,4.220000,128.000000,...,0.000000,2912000.000000,2014,1237192320000.000000,36881936.000000,432000000.000000,2863.871111,0.085375,1861516222222.222168,55493653.703704
3,1,Armenia,2511,Wheat and products,2017,vegetal,547.000000,1.000000,4.280000,105.000000,...,0.000000,2912000.000000,2014,1226563520000.000000,36520556.800000,428000000.000000,2865.802617,0.085328,1567594031401.869141,46674636.844860
4,1,Armenia,2511,Wheat and products,2018,vegetal,571.000000,1.000000,4.400000,118.000000,...,0.000000,2912000.000000,2014,1223374880000.000000,36403640.000000,428000000.000000,2858.352523,0.085055,1632119290841.121338,48566538.411215


# Task5

Load food security data to get malnutrition information and do necessary cleanings

In [20]:
food_security = pd.read_csv("food_security.csv")
food_security.head()

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,FS,Suite of Food Security Indicators,2,Afghanistan,6121,Value,21010,Average dietary energy supply adequacy (percen...,20132015,2013-2015,%,96,F,FAO estimate,nan
1,FS,Suite of Food Security Indicators,2,Afghanistan,6122,Value,21011,Average value of food production (constant 200...,20132015,2013-2015,I$ per person,104,F,FAO estimate,nan
2,FS,Suite of Food Security Indicators,2,Afghanistan,6128,Value,22000,Dietary energy supply used in the estimation o...,20132015,2013-2015,kcal/capita/day,2040,F,FAO estimate,nan
3,FS,Suite of Food Security Indicators,2,Afghanistan,6121,Value,21012,Share of dietary energy supply derived from ce...,20132015,2013-2015,%,74,F,FAO estimate,nan
4,FS,Suite of Food Security Indicators,2,Afghanistan,6123,Value,21013,Average protein supply (g/cap/day) (3-year ave...,20132015,2013-2015,g/capita/day,56,F,FAO estimate,nan


In [21]:
## find a country with malnutrition rate over 10percent
malnutrition_data=food_security.loc[(food_security['Item'] == 'Prevalence of undernourishment (percent) (3-year average)')]
malnutrition_data=malnutrition_data.replace({'Value': {'<': ''}}, regex=True)
malnutrition_data = malnutrition_data[malnutrition_data['Value'].notna()]
#malnutrition_data.sort_values("Value")
malnutrition_data.Value= malnutrition_data.Value.astype(float)
malnutrition_data=malnutrition_data.loc[(malnutrition_data['Value'] >10)]
malnutrition_data=malnutrition_data.drop(['Domain Code','Domain', 'Area Code', 'Element Code', 'Element', 'Item Code','Year Code', 'Year', 'Unit','Flag','Flag Description','Note'], axis=1)
malnutrition_data = malnutrition_data.rename(columns = {'Area': 'country'}, inplace = False)
malnutrition_data.head()

,country,Item,Value
8,Afghanistan,Prevalence of undernourishment (percent) (3-ye...,26.400000
93,Angola,Prevalence of undernourishment (percent) (3-ye...,23.100000
291,Bangladesh,Prevalence of undernourishment (percent) (3-ye...,14.500000
459,Bolivia (Plurinational State of),Prevalence of undernourishment (percent) (3-ye...,17.300000
505,Botswana,Prevalence of undernourishment (percent) (3-ye...,19.400000


In [28]:
df=df.rename(columns={"year_x": "year"})
exported=df.loc[(df['year'] ==2018)]
exported=exported[["country", "item","export_quantity","year"]]
exported = exported[exported['export_quantity'].notna()]
## finding 25 most exported items by countries with mal-nutrition rate over 10% - using left join
exported_items_top_25= pd.merge(malnutrition_data,exported,on='country',how='left')
exported_items_top_25=exported_items_top_25.groupby('item')['export_quantity'].sum()
exported_items_top_25= pd.DataFrame(data=exported_items_top_25)
exported_items_top_25=exported_items_top_25.reset_index()
exported_items_top_25=exported_items_top_25.nlargest(25,['export_quantity'])
exported_items_top_25

,item,export_quantity
73,Rice and products,25213.000000
4,Bananas,14008.000000
83,Sugar (Raw Equivalent),9210.000000
94,Wheat and products,3730.000000
93,"Vegetables, Other",3299.000000
62,Pelagic Fish,2455.560000
41,Maize and products,2417.000000
17,Cocoa Beans and products,2151.000000
66,Pineapples and products,2042.000000
32,"Fruits, Other",2038.000000


In [29]:
import_items=df.loc[(df['year'] ==2018)]
#import_items=import_items[["country", "item","import_quantity","year"]]
import_items_top_25=import_items.merge(exported_items_top_25, on='item', how='left', indicator=True)
import_items_top_25=import_items_top_25.nlargest(200,['import_quantity'])
import_items_top_25=import_items_top_25.replace({'_merge': {'both': 'True', 'left_only': 'False'}})
import_items_top_25

,country_code,country,item_code,item,year,origin,domestic_supply_quantity,export_quantity_x,fat_supply_quantity_gcapitaday,feed,...,year_y,food_supply_kcal,food_supply_kgprotein,food_supply_kg,ratio_kcalkg,protein_percentage,dom_sup_kcal,dom_sup_kgprot,export_quantity_y,_merge
2467,41,"China, mainland",2555,Soyabeans,2018,vegetal,105888.000000,178.000000,1.100000,9110.000000,...,2014,17367224140000.000000,1593698215.200000,5001000000.000000,3472.750278,0.318676,367722581430977.812500,33743954531.313255,nan,False
2454,41,"China, mainland",2532,Cassava and products,2018,vegetal,29783.000000,37.000000,0.010000,15600.000000,...,2014,3064804260000.000000,25540035.500000,2828000000.000000,1083.735594,0.009031,32276897197871.285156,268974143.315594,1088.000000,True
9186,138,Mexico,2514,Maize and products,2018,vegetal,40514.000000,1223.000000,11.030000,18622.000000,...,2014,43709927125000.000000,1126354303.000000,14813000000.000000,2950.781552,0.076038,119547963784665.500000,3080612855.717410,2417.000000,True
7039,110,Japan,2514,Maize and products,2018,vegetal,15819.000000,1.000000,0.260000,12010.000000,...,2014,3789316485000.000000,9824153.850000,1685000000.000000,2248.852513,0.005830,35574597908732.937500,92230439.022641,2417.000000,True
15261,237,Viet Nam,2514,Maize and products,2018,vegetal,14655.000000,82.000000,1.160000,12259.000000,...,2014,4150975640000.000000,103104878.800000,1402000000.000000,2960.752953,0.073541,43389834525106.992188,1077747502.720400,2417.000000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5421,90,Guinea,2805,Rice and products,2018,vegetal,3527.000000,0.000000,1.680000,915.000000,...,2014,4135236840000.000000,80669679.300000,2096000000.000000,1972.918340,0.038487,6958482984103.052734,135745209.394609,25213.000000,True
1380,21,Brazil,2659,"Alcohol, Non-Food",2018,vegetal,2727.000000,1342.000000,0.000000,0.000000,...,2014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1330.000000,True
6464,103,Iraq,2805,Rice and products,2018,vegetal,1676.000000,0.000000,0.640000,0.000000,...,2014,3673708940000.000000,70349910.200000,1660000000.000000,2213.077675,0.042379,3709118182795.181152,71027981.623614,25213.000000,True
4150,68,France,2531,Potatoes and products,2018,vegetal,5972.000000,2729.000000,0.130000,440.000000,...,2014,2195112920000.000000,51138894.400000,3292000000.000000,666.802224,0.015534,3982142879173.754883,92770801.141191,1505.000000,True


## Question 1: considering only plant products, what portion of global domestic supply is used as food,feed,losses,other uses?

In [30]:
plant_data_2018=df[(df['year'] ==2018)& (df['origin'] =='vegetal')]
plant_data_2018=plant_data_2018.fillna(0)
global_supply_quantity= plant_data_2018['domestic_supply_quantity'].sum()
global_feed_food_losses_others=plant_data_2018['food']+plant_data_2018['feed']+plant_data_2018['losses']+plant_data_2018['other_uses']
plant_data_2018['global_feed_food_losses_others']=global_feed_food_losses_others
global_feed_food_losses_others=plant_data_2018['global_feed_food_losses_others'].sum()
global_supply_quantity/global_feed_food_losses_others

1.4012648858309764

## Question 2: How many humans on earth could be fed if all the plant-based food supply (crops), including food and feed, was used for human consumption? Result in Calories and Protein and Express as a percentage of world's population.

In [34]:
plant_data_total=df[(df['origin'] =='vegetal')]
plant_feed_capacity=plant_data_total[['country','year','item_code','item','food','feed','ratio_kcalkg','protein_percentage']]
plant_feed_capacity['kcal']=(plant_feed_capacity['food']+plant_feed_capacity['feed'])*1000000*plant_feed_capacity['ratio_kcalkg']
plant_feed_capacity['protein_kg']=(plant_feed_capacity['food']+plant_feed_capacity['feed'])*1000000*plant_feed_capacity['protein_percentage']
plant_feed_capacity=plant_feed_capacity.replace([np.inf, -np.inf], np.nan)
plant_feed_capacity['kcal']=plant_feed_capacity['kcal'].fillna(0)
plant_feed_capacity['protein_kg']=plant_feed_capacity['protein_kg'].fillna(0)
plant_feed_capacity=plant_feed_capacity.groupby(['year'])['kcal','protein_kg'].sum()
plant_feed_capacity=plant_feed_capacity.reset_index()
plant_feed_capacity

C:\Users\jjanbeb\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jjanbeb\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,year,kcal,protein_kg
0,2014,8703058564109355.000000,191073508643.792419
1,2015,8905403661768202.000000,197036974654.184662
2,2016,8916283073007392.000000,196206588100.445862
3,2017,9039091533579762.000000,201118985269.119507
4,2018,9012830688379056.000000,198806676817.483765


In [48]:
#Daily Carlories and protein requirements for average human
daily_carl=2500
avg_weight=62
daily_protein=0.008
#Creating new column calculating amount of people that could be fed in 2014 and 2018
plant_feed_capacity['pop_fed_bykcal']=plant_feed_capacity['kcal']/daily_carl/365
plant_feed_capacity['pop_fed_bykcal']=plant_feed_capacity['pop_fed_bykcal'].round(0)
plant_feed_capacity['pop_fed_byprotein']=plant_feed_capacity['protein_kg']/(avg_weight*daily_protein)/365
plant_feed_capacity['pop_fed_byprotein']=plant_feed_capacity['pop_fed_byprotein'].round(0)
plant_popualtion=pd.merge(population,plant_feed_capacity, on='year',how='left')
plant_feed_capacity['percentage_pop_fed_kcal']=100*plant_feed_capacity['pop_fed_bykcal']/plant_popualtion['population']
plant_feed_capacity['percentage_pop_fed_protein']=100*plant_feed_capacity['pop_fed_byprotein']/plant_popualtion['population']
plant_feed_capacity

,year,kcal,protein_kg,pop_fed_bykcal,pop_fed_byprotein,percentage_pop_fed_kcal,percentage_pop_fed_protein
0,2014,8703058564109355.000000,191073508643.792419,9537598426.000000,1055421502.000000,133.739304,14.799463
1,2015,8905403661768202.000000,197036974654.184662,9759346479.000000,1088361548.000000,135.306385,15.089357
2,2016,8916283073007392.000000,196206588100.445862,9771269121.000000,1083774791.000000,133.970933,14.859310
3,2017,9039091533579762.000000,201118985269.119507,9905853735.000000,1110909110.000000,134.338556,15.065630
4,2018,9012830688379056.000000,198806676817.483765,9877074727.000000,1098136748.000000,132.522454,14.733894


## Question3: How many humans could be fed with the global food supply? Give the results in terms of calories and protein. Express these two results as a percentage of the world's population.

In [70]:
global_feed_capacity=df
global_feed_capacity.head()
global_feed_capacity=global_feed_capacity[['country','year','item_code','item','food','feed','ratio_kcalkg','protein_percentage']]
global_feed_capacity['kcal']=(global_feed_capacity['food']+global_feed_capacity['feed'])*1000000*global_feed_capacity['ratio_kcalkg']
global_feed_capacity['protein_kg']=(global_feed_capacity['food']+global_feed_capacity['feed'])*1000000*global_feed_capacity['protein_percentage']
global_feed_capacity=global_feed_capacity.replace([np.inf, -np.inf], np.nan)
global_feed_capacity['kcal']=global_feed_capacity['kcal'].fillna(0)
global_feed_capacity['protein_kg']=global_feed_capacity['protein_kg'].fillna(0)
global_feed_capacity=global_feed_capacity.groupby(['year'])['kcal','protein_kg'].sum()
global_feed_capacity=global_feed_capacity.reset_index()


C:\Users\jjanbeb\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\jjanbeb\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [71]:
#Daily Carlories and protein requirements for average human
daily_carl=2500
avg_weight=62
daily_protein=0.008
#Creating new column calculating amount of people that could be fed in 2014 and 2018
global_feed_capacity['pop_fed_bykcal']=global_feed_capacity['kcal']/daily_carl/365
global_feed_capacity['pop_fed_bykcal']=global_feed_capacity['pop_fed_bykcal'].round(0)
global_feed_capacity['pop_fed_byprotein']=global_feed_capacity['protein_kg']/(avg_weight*daily_protein)/365
global_feed_capacity['pop_fed_byprotein']=global_feed_capacity['pop_fed_byprotein'].round(0)
global_popualtion=pd.merge(population,global_feed_capacity, on='year',how='left')
global_feed_capacity['percentage_pop_fed_kcal']=100*global_feed_capacity['pop_fed_bykcal']/global_popualtion['population']
global_feed_capacity['percentage_pop_fed_protein']=100*global_feed_capacity['pop_fed_byprotein']/global_popualtion['population']
global_feed_capacity

,year,kcal,protein_kg,pop_fed_bykcal,pop_fed_byprotein,percentage_pop_fed_kcal,percentage_pop_fed_protein
0,2014,10132246051343626.000000,279411076237.962280,11103831289.000000,1543366528.000000,155.701530,21.641587
1,2015,10350763920915920.000000,285999474835.060669,11343302927.000000,1579758478.000000,157.266813,21.902226
2,2016,10373989391427984.000000,285790184620.062561,11368755497.000000,1578602434.000000,155.873588,21.643743
3,2017,10511880889407266.000000,292014570789.501221,11519869468.000000,1612983710.000000,156.227082,21.874531
4,2018,10514836195553596.000000,290759073323.043457,11523108160.000000,1606048792.000000,154.607575,21.548640


## Question4: what proportion of the world's population is considered undernourished?

In [51]:
malnutrition_data.head()

,country,Item,Value
8,Afghanistan,Prevalence of undernourishment (percent) (3-ye...,26.400000
93,Angola,Prevalence of undernourishment (percent) (3-ye...,23.100000
291,Bangladesh,Prevalence of undernourishment (percent) (3-ye...,14.500000
459,Bolivia (Plurinational State of),Prevalence of undernourishment (percent) (3-ye...,17.300000
505,Botswana,Prevalence of undernourishment (percent) (3-ye...,19.400000


In [58]:
malnutiriton_population=pd.merge(malnutrition_data,population_uncleaned,on='country')
mulnutrition_population_sum=malnutiriton_population.population.sum()
population_sum=population.population.sum()
malnutrition_population_percentage=mulnutrition_population_sum/population_sum
malnutrition_population_percentage

0.3425212162337748

## Question5:Considering the 25 items most exported by the countries with a high rate of undernutrition, which three of them:Have the greatest other_uses to domestic_supply_quantity ratio and what are they used for?Have the greatest feed to (food+feed) ratio and what are they used for?

In [59]:
group_item=df.groupby(['item'])['food','feed','domestic_supply_quantity','other_uses'].sum().reset_index()
top_three=pd.merge(group_item,exported_items_top_25,on='item')
top_three['ratio_other_domsupply']=top_three['other_uses']/top_three['domestic_supply_quantity']
top_three['ratio_feed_foofeed']=top_three['food']/(top_three['food']+top_three['feed'])
ratio_other_domsupply=top_three.sort_values(by='ratio_other_domsupply', ascending=False)
ratio_feed_foofeed=top_three.sort_values(by='ratio_feed_foofeed', ascending=False)
ratio_other_domsupply.head(3)


,item,food,feed,domestic_supply_quantity,other_uses,export_quantity,ratio_other_domsupply,ratio_feed_foofeed
0,"Alcohol, Non-Food",23.000000,0.000000,74644.000000,75151.000000,1330.000000,1.006792,1.000000
15,Palm Oil,78575.000000,204.000000,299634.000000,162216.000000,2028.000000,0.541380,0.997410
6,Coconut Oil,6043.000000,0.000000,13675.000000,3803.000000,975.000000,0.278099,1.000000


In [60]:
ratio_feed_foofeed.head(3).head()

,item,food,feed,domestic_supply_quantity,other_uses,export_quantity,ratio_other_domsupply,ratio_feed_foofeed
0,"Alcohol, Non-Food",23.000000,0.000000,74644.000000,75151.000000,1330.000000,1.006792,1.000000
13,Nuts and products,74911.000000,0.000000,77064.000000,180.000000,1560.000000,0.002336,1.000000
17,Pineapples and products,105974.000000,0.000000,117248.000000,0.000000,2042.000000,0.000000,1.000000


## Question6: Taking only gains for food and feed into account, what proprtion is used for feed? (kg)

In [61]:
cereal=df[(df['item'] =='Cereals, Other')]
cereal=cereal[["feed", "food"]]
cereal['feed_and_food']=cereal['food']+cereal['feed']
cereal_feed_and_food=cereal['feed_and_food'].sum()
cereal_feed=cereal['feed'].sum()
cereal_feed/cereal_feed_and_food

0.7482910867567628

## Question7: How many tons of grains (cereals) could be released if the US reduced its production of animal products by 10%? Convert this quantity to kcal, and the number of potentially fed humans.

In [64]:
pd.set_option('float_format', '{:f}'.format)
us_animal_production=df[(df['country'] =='United States of America')& (df['origin'] =='animal')]
us_animal_production=us_animal_production[["country", "item","food_supply_kcal","production",'population','feed']]
us_animal_production=us_animal_production.fillna(0)
us_cereals_production=df[(df['country'] =='United States of America')& (df['item'] =='Cereals, Other')]
us_cereals_production=us_cereals_production[["year","country", "item","food_supply_kcal","production",'population','feed','ratio_kcalkg']]
us_cereals_production
reduced_animal_production_kg=us_animal_production.production.sum()*0.1*1000
release_cereals_kg=reduced_animal_production_kg*0.7478258288563712
feed_quantity_kcal=release_cereals_kg*us_cereals_production.ratio_kcalkg.sum()
feed_quantity_kcal
possible_fed_human=feed_quantity_kcal/2500
possible_fed_human

402376245.2099326

## Question8:In Thailand, what proportion of cassava is exported? What is the proportion of undernutrition?

In [72]:
thailand_data=df[(df['country'] =='Thailand')& (df['item'] =='Cassava and products')]
cassava_export_ratio=thailand_data.export_quantity.sum()/thailand_data.production.sum()
cassava_export_ratio


0.904748246460327

In [73]:
thailand_malnutrition=food_security[(food_security['Area'] =='Thailand')& (food_security['Item'] =='Prevalence of undernourishment (percent) (3-year average)')]     
thailand_malnutrition

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
3819,FS,Suite of Food Security Indicators,216,Thailand,6121,Value,210041,Prevalence of undernourishment (percent) (3-ye...,20132015,2013-2015,%,8.8,F,FAO estimate,nan
